<a href="https://colab.research.google.com/github/lingchen0331/wallybob/blob/master/MultiTCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Use the tensorflow 2.0 backend
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
import warnings
import numpy as np
import tensorflow as tf
import pandas as pd
import json
from tensorflow.keras import datasets, layers, models, Input
from tensorflow.keras import Model
import numpy as np
import tensorflow.keras as keras

In [0]:
# Hyper-parameters
seq_len = 200
label_len = 20
seq_num = 1610
input_channels = 1

FILE_PATH = "/content/drive/My Drive/TCN/"

In [0]:
class MultiTCN(Model):
  def __init__(self):
      super(MultiTCN, self).__init__()
      self.conv1 = layers.Conv2D(32, 3, activation='relu', input_shape=(400, 400, 1), data_format="channels_last")
      self.batch_norm = layers.BatchNormalization()
      self.activation = layers.Activation('relu')
      #self.dropout = layers.SpatialDropout1D(rate=0.1)
      
      self.conv2 = layers.Conv2D(32, 3, activation='relu', dilation_rate=2, input_shape=(400, 400, 1))
      self.flat = layers.Flatten()
      self.dense_1 = layers.Dense(500, activation='relu')
      self.dense_2 = layers.Dense(20)
      self.softmax = layers.Activation('softmax')
      
  def model(self):
      x = Input(shape=(400, 400, 1))
      return Model(inputs=[x], outputs=self.call(x))
      
  def call(self, inputs):
      inputs_padding_1 = tf.pad(inputs, [[0, 0], [2, 0], [2, 0], [0, 0]])
      conv1_op = self.conv1(inputs_padding_1)
      batch_norm1 = self.batch_norm(conv1_op)
      activation_1 = self.activation(batch_norm1)
      #dropout_1 = self.dropout(activation_1)
    
      # Second Layer
      padding_2 = tf.pad(activation_1, [[0, 0], [4, 0], [4, 0], [0, 0]])
      conv2_op = self.conv2(padding_2)
      batch_norm2 = self.batch_norm(conv2_op)
      activation_2 = self.activation(batch_norm2)
      #dropout_2 = self.dropout(activation_2)
      fc_1 = self.dense_1(activation_2)
      fc_2 = self.dense_2(fc_1)
      activation_3 = self.softmax(fc_2)
      return activation_3

In [0]:
def load_json(filename):
    """Load json files in a given directory"""
    data = []
    with open(filename) as f:
        for line in f:
            data.append(json.loads(line))

    data = pd.DataFrame(data)
    return data


def count_number(test_list, l, r): 
  c = 0 
  # traverse in the list1 
  for x in test_list: 
    # condition check 
    if x>= l and x<= r: 
      c+= 1 
  return c


def zero_detection(myList):
    list_index = list()
    for i in range(myList.shape[1]):
        col_vec = myList[:, i].tolist()
        index = next((j for j, x in enumerate(col_vec) if x), None)
        list_index.append(index)
    return list_index


def pad_sequence(reply_list):
    earliest_time = int(reply_list[0][0])
    latest_time = int(reply_list[-1][-1])
    
    # Get the time duration based on the earlist and latest time
    time_duration = []
    for i in range(earliest_time, latest_time, 300):
        time_duration.append(i)
    
    time_duration_mod = []
    for i in range(len(time_duration)-1):
        time_duration_mod.append((time_duration[i], time_duration[i+1]))
    
    # Generate a dataframe storing all the sequence
    df = pd.DataFrame([])
    
    for index, seq in enumerate(reply_list):
        time_series = []
        for k in time_duration_mod:
            time_series.append(count_number(seq, k[0], k[1]))
    
        df['{}'.format(index)] = time_series
        print("Done, {}".format(index))
    
    print("Finished padding sequence...")
    return np.asarray(df)
  
  
def generating_dataset():
    # Read stream data
    main_stream = load_json(FILE_PATH+'data/james.json')  # All main stream data.
    # reply_stream = dh.load_json('james_reply.json') # All reply stream data.
    reply_stream = [load_json(FILE_PATH+"data/james/{}.json".format(i)) for i in range(len(main_stream))]
    
    reply_list = []
    main_time = [float(i) for i in main_stream.created]
    # Get the reply time series
    for i,x in enumerate(reply_stream):
        try:
            temp_list = [main_time[i]]
            temp_list = temp_list + list(x.created_at)
            reply_list.append(temp_list)
        except:
            reply_list.append([main_time[i]])
    
    return reply_list


def get_data(origin_data, seq_len, label_len):
    features, labels_temp, labels = [], [], []
    index_list = zero_detection(origin_data)
    truncated_index = index_list[seq_len-1:1605]
    for i, x in enumerate(truncated_index):
        features.append(origin_data[x-seq_len+1:x+1, i:i+seq_len])
        labels_temp.append(origin_data[x:x+label_len, i+seq_len])
        
    zero_indices = []
    for i, x in enumerate(labels_temp):
        if np.count_nonzero(x) == 0:
            zero_indices.append(i)
    
    features = [v for i,v in enumerate(features) if i not in zero_indices]
    labels_temp = [v for i,v in enumerate(labels_temp) if i not in zero_indices]
    
    for i in labels_temp:
        labels.append(next((j for j, x in enumerate(i) if x), None))
    
    features = np.array([np.reshape(i, i.shape+(-1,)).astype('float64') for i in features])
    labels = keras.utils.to_categorical(labels)
    
    return features, labels
  
  
def building_dataset(train_x, train_y, valid_x, valid_y):
  # Building datasets
  train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
  train_dataset = train_dataset.batch(1)

  valid_dataset = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
  valid_dataset = valid_dataset.batch(1)
    
  print("Finish constructing dataset...")
  return train_dataset, valid_dataset


def transform_dataShape(input_matrix):
    # Transform datashape
    for i in range(len(input_matrix)):
        input_matrix[i] = np.reshape(input_matrix[i], input_matrix[i].shape+(-1,)).astype('float32')
      
    return input_matrix

In [0]:
tt = generating_dataset()

In [0]:
# Getting original data
origin_data = pad_sequence(tt)

Done, 0
Done, 1
Done, 2
Done, 3
Done, 4
Done, 5
Done, 6
Done, 7
Done, 8
Done, 9
Done, 10
Done, 11
Done, 12
Done, 13
Done, 14
Done, 15
Done, 16
Done, 17
Done, 18
Done, 19
Done, 20
Done, 21
Done, 22
Done, 23
Done, 24
Done, 25
Done, 26
Done, 27
Done, 28
Done, 29
Done, 30
Done, 31
Done, 32
Done, 33
Done, 34
Done, 35
Done, 36
Done, 37
Done, 38
Done, 39
Done, 40
Done, 41
Done, 42
Done, 43
Done, 44
Done, 45
Done, 46
Done, 47
Done, 48
Done, 49
Done, 50
Done, 51
Done, 52
Done, 53
Done, 54
Done, 55
Done, 56
Done, 57
Done, 58
Done, 59
Done, 60
Done, 61
Done, 62
Done, 63
Done, 64
Done, 65
Done, 66
Done, 67
Done, 68
Done, 69
Done, 70
Done, 71
Done, 72
Done, 73
Done, 74
Done, 75
Done, 76
Done, 77
Done, 78
Done, 79
Done, 80
Done, 81
Done, 82
Done, 83
Done, 84
Done, 85
Done, 86
Done, 87
Done, 88
Done, 89
Done, 90
Done, 91
Done, 92
Done, 93
Done, 94
Done, 95
Done, 96
Done, 97
Done, 98
Done, 99
Done, 100
Done, 101
Done, 102
Done, 103
Done, 104
Done, 105
Done, 106
Done, 107
Done, 108
Done, 109
Done, 110


In [0]:
features, labels = get_data(origin_data, seq_len, label_len)

In [0]:
# Splitting training, validation, and testing data
train_x = features[0:1000]
train_y = labels[0:1000]

valid_x = features[1000:1170]
valid_y = labels[1000:1170]

test_x = features[1170:]
test_y = labels[1170:]

In [0]:
train_y[0].shape

(20,)

In [0]:
train_dataset, valid_dataset = building_dataset(train_x, train_y, valid_x, valid_y)

Finish constructing dataset...


In [0]:
# Create an instance of the model
model = MultiTCN()
model.compile(optimizer='rmsprop', 
              loss=keras.losses.categorical_crossentropy, 
              metrics=['accuracy'])

In [0]:
model.fit(train_dataset, validation_data=valid_dataset, epochs=100, verbose=1)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/100
1000/1000 [==============================] - 43s 43ms/step - loss: 2.5193 - accuracy: 0.1923 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
1000/1000 [==============================] - 40s 40ms/step - loss: 2.4510 - accuracy: 0.2089 - val_loss: 2.3686 - val_accuracy: 0.2000
Epoch 3/100
1000/1000 [==============================] - 40s 40ms/step - loss: 2.4448 - accuracy: 0.2090 - val_loss: 2.3887 - val_accuracy: 0.2000
Epoch 4/100
1000/1000 [==============================] - 40